#Setup

In [ ]:
!pip install pandas
!pip install rtree
!pip install pygeos

In [ ]:
import rtree
import pygeos

In [ ]:
!pip install fiona
!pip install pyproj
!pip install gdal
!pip install shapely
!pip install basemap
!pip install geopandas
!pip install networkx

# Install packages
!pip install folium
!pip install sodapy -q
!pip install adjustText
!pip install plotly --upgrade
!pip install contextily
!pip install mapclassify
!pip install chart-studio

In [ ]:
#hosted runtime

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

drivepath = '/content/drive/My Drive/Columbia/3 Fall 22/RA/Detroit/'

In [ ]:
#local runtime

#drivepath = '/Users/kirthi/kb3185@columbia.edu - Google Drive/My Drive/Columbia/3 Fall 22/RA/Detroit'

In [ ]:
import warnings
import os
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from shapely.geometry import shape
from shapely.geometry import Point


from urllib.parse import urlencode
import urllib.request, json 

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import random
from random import randint

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as py
import plotly.express as px
from sklearn.preprocessing import minmax_scale
py.init_notebook_mode(connected=False)

In [ ]:
import geopandas as gpd
from pyproj import CRS
from shapely import wkt

In [ ]:
# import more packages
import plotly.io as pio
pio.renderers.default = "colab"
access_token = 'pk.eyJ1IjoiYmtpcnRoaSIsImEiOiJja3VpdzFiMnoycnYyMzBtYXM1aW1uemd0In0.knHw7eYrN8DvrrBHZugW8Q'
px.set_mapbox_access_token(access_token) # for interactive plotly viz

import plotly.graph_objects as go
import chart_studio.plotly as py
import chart_studio.tools as tls

#Data Cleaning

In [ ]:
path = drivepath + 'Data/NHGIS/nhgis0001_shape/nhgis0001_shapefile_tl2020_us_zcta_2020/US_zcta_2020.shp'
zcta = gpd.read_file(path)
zcta = zcta[['GISJOIN','ZCTA5CE20','ALAND20','AWATER20','INTPTLAT20','INTPTLON20','geometry']]
zcta['ZCTA5CE20'] = zcta['ZCTA5CE20'].astype(int)

In [ ]:
path = drivepath + 'Data/Area of Interest - ZCTAs.csv'
dt = pd.read_csv(path)
dt.rename(columns={'ZCTA':'ZCTA5CE20', '"City" Name':'City Name'},inplace=True)

dt = dt.merge(zcta,on='ZCTA5CE20')
df = gpd.GeoDataFrame(dt, geometry='geometry')
df.head()

In [ ]:
path = drivepath + 'Data/NHGIS/nhgis0001_shape/nhgis0001_shapefile_tl2000_us_tract_1970/US_tract_1970.shp'
ct = gpd.read_file(path)

In [ ]:
path = drivepath + 'Data/NHGIS/nhgis0001_csv/nhgis0001_ds107_1980_tract.csv' #1980 APAN
df_1980 = pd.read_csv(path)

df_1980 = df_1980[['GISJOIN','STATE','DGJ031',
       'DGJ032', 'DGJ033', 'DGJ034', 'DGJ035', 'DGJ036', 'DGJ037', 'DGJ038',
       'DGJ039', 'DGJ040']]

df_1980 = ct.merge(df_1980,on='GISJOIN')
df_1980 = df_1980.loc[df_1980['STATE']=='Michigan']

fdf = df_1980.iloc[:,13:].copy()
fdf.columns.values[:] = df_1980.iloc[:,8:13].columns.values[:]

df_1980.iloc[:,8:13] = df_1980.iloc[:,8:13].add(fdf, fill_value=0)
df_1980 = df_1980.iloc[:,:13]

df_1980_p = df_1980.copy()
# change geometry 
df_1980_p['geometry'] = df_1980_p['geometry'].centroid

df_1980_n = df_1980_p.sjoin(zcta, how="left")
df_1980 = df_1980_n.iloc[:,8:16].drop(columns='index_right').rename(columns={'GISJOIN_right':'GISJOIN'}).groupby(['ZCTA5CE20']).sum()
df_1980 = dt.merge(df_1980,on='ZCTA5CE20')
df_1980.rename(columns={'DGJ031':'Under 5 years','DGJ032':'5-14 years','DGJ033':'15-59 years','DGJ034':'60-64 years','DGJ035':'65 years and over'},inplace=True)
df_1980 = gpd.GeoDataFrame(df_1980, geometry='geometry')

In [ ]:
gpd_path = drivepath + '/Data/ZCTA Boundaries/tl_2021_us_zcta520/tl_2021_us_zcta520.shp'
gdf = gpd.read_file(gpd_path)
gdf = gdf[['ZCTA5CE20','ALAND20','AWATER20','INTPTLAT20','INTPTLON20','geometry']]

gdf['ZCTA5CE20'] = gdf['ZCTA5CE20'].astype(int)

path = drivepath + 'Data/Area of Interest - ZCTAs.csv'
dt = pd.read_csv(path)
dt.rename(columns={'ZCTA':'ZCTA5CE20', '"City" Name':'City Name'},inplace=True)

dt = dt.merge(gdf,on='ZCTA5CE20')
df = gpd.GeoDataFrame(dt, geometry='geometry').set_index('ZCTA5CE20')

df_1980 = df.merge(df_1980.set_index('ZCTA5CE20').iloc[:,7:], on = 'ZCTA5CE20').reset_index()

In [ ]:
def get_csv(year):
  path = drivepath + 'Data/NHGIS/nhgis0001_csv/nhgis0001_ts_geog2010_data'+str(year)+'_zcta.csv' #1990 APAN
  df_1990 = pd.read_csv(path)
  df_1990 = df_1990.loc[:, ~(df_1990.columns.str.endswith('U') | df_1990.columns.str.endswith('L'))]
  fdf = df_1990.iloc[:,26:].copy()
  fdf.columns.values[:] = df_1990.iloc[:,4:26].columns.values[:]

  df_1990.iloc[:,4:26] = df_1990.iloc[:,4:26].add(fdf, fill_value=0)
  df_1990 = df_1990.iloc[:,:26]

  df_1990.drop(columns={'GEOGYEAR','DATAYEAR','GISJOIN'},inplace=True)
  df_1990.rename(columns = lambda x : str(x)[:-4],inplace=True)
  df_1990.rename(columns={'Z':'ZCTA5CE20'},inplace=True)
  df_1990 = dt.merge(df_1990,on='ZCTA5CE20')
  df_1990 = gpd.GeoDataFrame(df_1990, geometry='geometry')

  df_1990['CX7AB'] = df_1990['CX7AB'] + df_1990['CX7AC']
  df_1990['CX7AP'] = df_1990['CX7AP'] + df_1990['CX7AQ']
  df_1990['CX7AR'] = df_1990['CX7AR'] + df_1990['CX7AS'] + df_1990['CX7AT'] + df_1990['CX7AU'] + df_1990['CX7AV']
  df_1990['CX7AD'] = df_1990.iloc[:,11:23].sum(axis=1)
  df_1990 = df_1990[['ZCTA5CE20','City Name','ALAND20','AWATER20','INTPTLAT20','INTPTLON20','geometry','CX7AA','CX7AB','CX7AD','CX7AP','CX7AR']]
  df_1990.rename(columns={'CX7AA':'Under 5 years','CX7AB':'5-14 years','CX7AD':'15-59 years','CX7AP':'60-64 years','CX7AR':'65 years and over'},inplace=True)
  
  return df_1990

In [ ]:
df_1990 = get_csv(1990) #1990 APAN
df_2000 = get_csv(2000) #2000 APAN
df_2010 = get_csv(2010) #2010 APAN

In [ ]:
path = drivepath + 'Data/NHGIS/nhgis0001_csv/nhgis0002_ds250_20205_zcta_E.csv' #2020 
df_2020 = pd.read_csv(path)
df_2020.rename(columns={'ZCTAA':'ZCTA5CE20'},inplace=True)
df_2020.set_index('ZCTA5CE20', inplace=True)
df_2020 = df_2020.iloc[:,39:]

fdf = df_2020.iloc[:,31:].copy()
fdf.columns.values[:] = df_2020.iloc[:,:31].columns.values[:]

df_2020.iloc[:,:31] = df_2020.iloc[:,:31].add(fdf, fill_value=0)
df_2020 = df_2020.iloc[:,1:31]

fdf = df_2020.iloc[:,15:].copy()
fdf.columns.values[:] = df_2020.iloc[:,:15].columns.values[:]

df_2020.iloc[:,:15] = df_2020.iloc[:,:15].add(fdf, fill_value=0)
df_2020 = df_2020.iloc[:,:15]

df_2020.reset_index(inplace=True)
df_2020 = dt.merge(df_2020,on='ZCTA5CE20')
df_2020 = gpd.GeoDataFrame(df_2020, geometry='geometry')

df_2020['AMZ3E004'] = df_2020['AMZ3E004'] + df_2020['AMZ3E005'] 
df_2020['AMZ3E013'] = (0.5*df_2020['AMZ3E013'])
df_2020['AMZ3E006'] = df_2020.iloc[:,12:20].sum(axis=1)
df_2020['AMZ3E014'] = df_2020['AMZ3E014'] + df_2020['AMZ3E015']  + df_2020['AMZ3E016'] 

df_2020 = df_2020[['ZCTA5CE20','ALAND20','AWATER20','INTPTLAT20','INTPTLON20','geometry','AMZ3E003','AMZ3E004','AMZ3E006','AMZ3E013','AMZ3E014']]
df_2020.rename(columns={'AMZ3E003':'Under 5 years','AMZ3E004':'5-14 years','AMZ3E006':'15-59 years','AMZ3E013':'60-64 years','AMZ3E014':'65 years and over'},inplace=True)
  

In [ ]:
filename = 'Outputs/ZCTA Census Data/1980_DEC_Census.csv'
path = drivepath+filename
df_1980.to_csv(path)

filename = 'Outputs/ZCTA Census Data/1990_DEC_Census.csv'
path = drivepath+filename
df_1990.to_csv(path)

filename = 'Outputs/ZCTA Census Data/2000_DEC_Census.csv'
path = drivepath+filename
df_2000.to_csv(path)

filename = 'Outputs/ZCTA Census Data/2010_DEC_Census.csv'
path = drivepath+filename
df_2010.to_csv(path)

filename = 'Outputs/ZCTA Census Data/2020_DEC_Census.csv'
path = drivepath+filename
df_2020.to_csv(path)

In [ ]:
k = 1980
for i in [df_1980,df_1990,df_2000,df_2010,df_2020]:
  i['Majority']=i.iloc[:,7:].idxmax(axis=1)
  i['Year'] = k
  k = k+ 10

In [ ]:
pdlist = [df_1980,df_1990,df_2000,df_2010,df_2020]
new_df = pd.concat(pdlist)
new_df

In [ ]:
filename = 'Outputs/ZCTA Census Data/All_DEC_Census.csv'
path = drivepath+filename
new_df.to_csv(path)